In [7]:
import geopandas as gpd
import numpy as np
import os
from my_function import filter_classes,supprimer_dossier_non_vide

In [9]:
# Paramètrage des paths
racine = r"C:\Users\orabo\Documents\GitHub\Projet Teledec"    # Racine du projet
output_dir = os.path.join(racine, "output_masque")    # Dossier de sortie
os.makedirs(output_dir, exist_ok=True)       # Crée le dossier output temporaire
path_masque_traite = os.path.join(output_dir,'mask_traite.shp')    # Path pour le fichier shp masque traité 
path_f_vege = os.path.join(racine,"data/project/FORMATION_VEGETALE.shp")    # Path pour le fichier shp formation végétale

In [11]:
f_vege = gpd.read_file(path_f_vege)
L_mask = ['Lande',
          'Formation herbacée',
          'Forêt ouverte de conifères purs',
          'Forêt ouverte de feuillus purs',
          'Forêt ouverte sans couvert arboré',
          'Forêt ouverte à mélange de feuillus et conifères',
          'Forêt fermée sans couvert arboré']
## Utilise la fonction filter_classes

In [ ]:
def masque_shp(path_f_vege):
    """
    Permet la création du masque en format shp à partir d'un fichier formation végétale.

    Parameters:
        path_f_vege (str): Chemin du fichier pour accéder au fichier formation végétale

    Returns:
        None
    """
    f_vege = gpd.read_file(path_f_vege)    # Mettre le path en paramètre
    L_mask = ['Lande',
          'Formation herbacée',
          'Forêt ouverte de conifères purs',
          'Forêt ouverte de feuillus purs',
          'Forêt ouverte sans couvert arboré',
          'Forêt ouverte à mélange de feuillus et conifères',
          'Forêt fermée sans couvert arboré']
    

In [ ]:
filtered = filter_classes(f_vege, L_mask)

In [ ]:
# Construction du masque en format shp
f_vege = gpd.read_file(path_f_vege)
L_mask = ['Lande','Formation herbacée','Forêt ouverte de conifères purs','Forêt ouverte de feuillus purs','Forêt ouverte sans couvert arboré',
        'Forêt ouverte à mélange de feuillus et conifères','Forêt fermée sans couvert arboré']
ones = np.ones((24041,1),dtype=int)
f_vege['value'] = ones

for i,j in zip(f_vege['TFV'],range(len(f_vege['value']))):
    if i in L_mask:
        #f_vege['mask'][j]=0
        f_vege.loc[j,'value'] = 0

for i in range(len(f_vege['value'])):
    if f_vege['value'][i] == 1:
        f_vege['Classe'] = 'Zone de forêt'
    else:
        f_vege['Classe'] = 'Zone hors forêt'

Masque = f_vege[['ID','Classe','value','geometry']]
Masque.loc[:,'value'] = Masque['value'].astype('uint8')
Masque.to_file(path_masque_traite) 

shp = gpd.read_file('/home/onyxia/work/data/project/emprise_etude.shp')

In [ ]:
for i,j in zip(f_vege['TFV'],range(len(f_vege['value']))):
    if i in L_mask:
        #f_vege['mask'][j]=0
        f_vege.loc[j,'value'] = 0

for i in range(len(f_vege['value'])):
    if f_vege['value'][i] == 1:
        f_vege['Classe'] = 'Zone de forêt'
    else:
        f_vege['Classe'] = 'Zone hors forêt'

Masque = f_vege[['ID','Classe','value','geometry']]
Masque.loc[:,'value'] = Masque['value'].astype('uint8')
Masque.to_file(path_masque_traite) 

shp = gpd.read_file('/home/onyxia/work/data/project/emprise_etude.shp')

In [ ]:
## Rasterization
my_folder = '/home/onyxia/work'
in_vector = path_masque_traite
out_image = os.path.join(my_folder, 'Depot_Git/results/data/img_pretraitees/masque_foret.tif')
field_name = 'value'  # field containing the numeric label of the classes

sptial_resolution = 10
xmin,ymin,xmax,ymax=shp.total_bounds
# xmin = shp['MINX'][0]
# ymin = shp['MINY'][0]
# xmax = shp['MAXX'][0]
# ymax = shp['MAXY'][0]

# Créer le répertoire de sortie si nécessaire
out_dir = os.path.dirname(out_image)
os.makedirs(out_dir, exist_ok=True)  # Crée les répertoires manquants

# define command pattern to fill with paremeters
cmd_pattern = ("gdal_rasterize -a {field_name} "
               "-tr {sptial_resolution} {sptial_resolution} "
               "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
               "{in_vector} {out_image}")

# fill the string with the parameter thanks to format function
cmd = cmd_pattern.format(in_vector=in_vector, xmin=xmin, ymin=ymin, xmax=xmax,
                         ymax=ymax, out_image=out_image, field_name=field_name,
                         sptial_resolution=sptial_resolution)

# execute the command in the terminal
os.system(cmd)

In [ ]:
supprimer_dossier_non_vide(output_dir)